In [1]:
from chat_solution.create_db import create_db

db = create_db()
print(db.retrieve("what is a llm?"))

Loading environment variables from /Users/jean.machado@getyourguide.com/prj/rag-workshop/.env


/Users/jean.machado@getyourguide.com/prj/rag-workshop/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Loading database from /tmp/embedding_db.pkl
Created 6 chunks of size 700 with overlap 200
Database saved to /tmp/embedding_db.pkl
Database saved successfully
['What are LLMs?\n\nLarge Language Models (LLMs) are trained on massive datasets of text to predict and generate language based on given prompts, learning patterns, structures, and relationships in text to produce human-like responses.\n\nHow do they work?\n\nWhat are the most known LLMs\nGpt-X series developed by Open-AI, they are proprietary and very powerful\nMistral Series: developed by Mistral AI, built by an eu company\nLLamma series: developed by Meta\n\nClosed source vs Open source LLMs\n\nClosed-source LLMs are proprietary, with code and models kept private, while open-source LLMs allow public access to the model architecture and often the training data, enabling more transparency and community-dr', 'What are LLMs?\n\nLarge Language Models (LLMs) are trained on massive datasets of text to predict and generate language bas

In [3]:
# User input and response handling
from chat_solution.rag import QuizRag

query2 = "what is an hallucination?"
rag = QuizRag()  
response = rag.query(query2)

Loading database from /tmp/embedding_db.pkl
Found 5 documents, first 500 characters: ['e LLMs are proprietary, with code and models kept private, while open-source LLMs allow public access to the model architecture and often the training data, enabling more transparency and community-driven improvements.\n\n\n\nWhat is RAG?\n\nRetrieval-Augmented Generation (RAG) combines language generation with real-time data retrieval, allowing models to access external sources or databases to provide more accurate, contextually relevant answers.\n\n\nWhat is a hallucination?\n A hallucination in AI refers to when a model generates information that sounds plausible but is factually incorrect or unsupported by the training data.\n\n\n\nPrompt engineering\nPrompt Tuning: Technique of adjusting the embedding', 'e LLMs are proprietary, with code and models kept private, while open-source LLMs allow public access to the model architecture and often the training data, enabling more transparency and commun

In [4]:
# Convert each text chunk to a LangChain Document
from langchain.schema import Document
from chat_solution.create_db import create_text_chunks_from_workshop_data

text_chunks = create_text_chunks_from_workshop_data()
# add all the content chunks to a list of LangChain Documents
langchain_docs = [
    Document(page_content=text, metadata={"source": f"chunk_{i+1}"})
    for i, text in enumerate(text_chunks)
]

print(len(langchain_docs))

Created 6 chunks of size 700 with overlap 200
6


In [5]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from langchain_mistralai import ChatMistralAI
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatMistralAI(model="mistral-large-latest"))
metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

generator_llm = LangchainLLMWrapper(ChatMistralAI(model="mistral-large-latest"))
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)

In [7]:
from ragas import EvaluationDataset

data = [
    {'user_input': 'what is a llm?',
     'reference': 'A large language model is a type of artificial intelligence model that is designed to understand and generate human language. These models are trained on vast amounts of text data and are capable of generating coherent and contextually appropriate responses to a wide range of questions.',
     'reference_contexts': ['A large language model is a type of artificial intelligence model that is designed to understand and generate human language. These models are trained on vast amounts of text data and are capable of generating coherent and contextually appropriate responses to a wide range of questions.']
     },
     {'user_input': 'how are some role models people the area of artificial intelligence?',
      'reference': 'Andrej Karpathy, a well-known figure in the field of artificial intelligence, is a role model for many people in the area. He is known for his work on deep learning and his contributions to the field of AI through his research and development of neural networks and deep learning algorithms.',
      'reference_contexts': ['Andrej Karpathy, a well-known figure in the field of artificial intelligence, is a role model for many people in the area. He is known for his work on deep learning and his contributions to the field of AI through his research and development of neural networks and deep learning algorithms.']
     },
     {'user_input': "what is an hallucination?",
      'reference': "An hallucination is a false perception or sensation that occurs in the absence of a corresponding external stimulus. It is often associated with mental health conditions such as schizophrenia or hallucinogenic substances.",
      'reference_contexts': ["An hallucination is a false perception or sensation that occurs in the absence of a corresponding external stimulus. It is often associated with mental health conditions such as schizophrenia or hallucinogenic substances."]
      }
]


# augment data with the llm response

for i, d in enumerate(data):
    response = rag.query(d['user_input'])
    data[i]['response'] = response
    data[i]['retrieved_contexts'] = rag.documents_retrieved


dataset = EvaluationDataset.from_list(data)




Found 5 documents, first 500 characters: ['What are LLMs?\n\nLarge Language Models (LLMs) are trained on massive datasets of text to predict and generate language based on given prompts, learning patterns, structures, and relationships in text to produce human-like responses.\n\nHow do they work?\n\nWhat are the most known LLMs\nGpt-X series developed by Open-AI, they are proprietary and very powerful\nMistral Series: developed by Mistral AI, built by an eu company\nLLamma series: developed by Meta\n\nClosed source vs Open source LLMs\n\nClosed-source LLMs are proprietary, with code and models kept private, while open-source LLMs allow public access to the model architecture and often the training data, enabling more transparency and community-dr', 'What are LLMs?\n\nLarge Language Models (LLMs) are trained on massive datasets of text to predict and generate language based on given prompts, learning patterns, structures, and relationships in text to produce human-like responses.\n\nHow

In [8]:

data

[{'user_input': 'what is a llm?',
  'reference': 'A large language model is a type of artificial intelligence model that is designed to understand and generate human language. These models are trained on vast amounts of text data and are capable of generating coherent and contextually appropriate responses to a wide range of questions.',
  'reference_contexts': ['A large language model is a type of artificial intelligence model that is designed to understand and generate human language. These models are trained on vast amounts of text data and are capable of generating coherent and contextually appropriate responses to a wide range of questions.'],
  'response': 'A Large Language Model (LLM) is trained on massive datasets of text to predict and generate language based on given prompts. It learns patterns, structures, and relationships in text to produce human-like responses.',
  'retrieved_contexts': ['What are LLMs?\n\nLarge Language Models (LLMs) are trained on massive datasets of te

In [9]:
from ragas.metrics import LLMContextPrecisionWithReference
from ragas.metrics import NonLLMContextRecall
from ragas.metrics import LLMContextRecall
from ragas.metrics import Faithfulness
from ragas.metrics import ResponseRelevancy
from ragas import evaluate

faithfulness = Faithfulness()
context_precision = LLMContextPrecisionWithReference()
context_recall = NonLLMContextRecall()
llm_context_recall = LLMContextRecall()
answer_relevancy = ResponseRelevancy()

eval_results = evaluate(
        dataset=dataset,
        metrics=[
                faithfulness,
                answer_relevancy,
                context_precision,
                context_recall,
                llm_context_recall
        ],
       raise_exceptions=False 
)

Evaluating: 100%|██████████| 15/15 [00:09<00:00,  1.60it/s]


In [10]:
evaluation_result_df = eval_results.to_pandas()
evaluation_result_df.iloc[:5]

,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,llm_context_precision_with_reference,non_llm_context_recall,context_recall
0,what is a llm?,[What are LLMs?\n\nLarge Language Models (LLMs...,[A large language model is a type of artificia...,A Large Language Model (LLM) is trained on mas...,A large language model is a type of artificial...,1.0,0.836305,1.0,0.0,1.0
1,how are some role models people the area of ar...,[ important role models in the field of AI & M...,"[Andrej Karpathy, a well-known figure in the f...","Based on the provided context, the important r...","Andrej Karpathy, a well-known figure in the fi...",1.0,0.927337,1.0,0.0,0.5
2,what is an hallucination?,"[e LLMs are proprietary, with code and models ...",[An hallucination is a false perception or sen...,Based on the context provided:\n\nA hallucinat...,An hallucination is a false perception or sens...,1.0,0.899427,0.0,0.0,0.0
